# NK plates analysis

## Preliminary settings

In [ ]:
library(ggplot2)
library(extrafont)
library(stringr)
library(heatmaply)
library(ggrepel)
library(reticulate)
library(gridExtra)
library(robustbase)

In [ ]:
# Load external Python functions
source_python("reticulate_functions.py")
# Load extra fonts
loadfonts()

In [ ]:
# Set this to true for faster compilation or false for more precise results and all outputs
TEST_MODE = T
# Number of permutations used for empirical p-value computations
PERM_NB_ITER = ifelse(TEST_MODE, 20, 2000)

In [ ]:
# Customize ggplot appearance

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}


cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
scale_fill_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)

## Import data

In [ ]:
NK = read.csv("./allImages.csv", header=T) 

In [ ]:
NK_P3B = read.csv("./allImages_NKP3B.csv", header=T)

Data for measurement B of plate 3 was handled separately and some column are not matching so a manual merging of the tables is needed.

In [ ]:
names(NK)[which(!(names(NK) %in% names(NK_P3B)))]
names(NK_P3B)[which(!(names(NK_P3B) %in% names(NK)))]

In [ ]:
commonNames = intersect(names(NK_P3B), names(NK))
# Common columns are in the same order:
all(names(NK_P3B)[names(NK_P3B) %in% commonNames] == names(NK)[names(NK) %in% commonNames])

In [ ]:
fields3B = str_extract(NK_P3B$URL_Actin, "r\\d{2}.*")
fieldsDuplicate = (str_extract(NK$URL_Actin, "Plate\\d") == "Plate3")&(str_extract(NK$URL_Actin, "r\\d{2}.*") %in% fields3B)

In [ ]:
NK = rbind(NK[!fieldsDuplicate, names(NK) %in% commonNames], 
      NK_P3B[, names(NK_P3B) %in% commonNames])

In [ ]:
NK$Plate <- as.factor(str_extract(NK$URL_Actin, "Plate."))
Position <- str_extract_all(NK$URL_Actin, '\\d{2}', simplify = T)
colnames(Position) <- c("Row", "Column", "Field")
NK <- cbind(NK, Position)

In [ ]:
wellAnnotation = read.csv("transferNK92.tsv", sep="\t", stringsAsFactors=F)

In [ ]:
wellAnnotation$Well <- sub("([A-H])(\\d$)", "\\10\\2", wellAnnotation$Well)

In [ ]:
getGene <- function(well){ # Which gene is targeted in a given well?
    return(wellAnnotation[wellAnnotation$Well == well, 2])
}
NK$Gene <- as.factor(sapply(NK$Metadata_Well, getGene))

In [ ]:
getGroup <- function(well){
    return(wellAnnotation[wellAnnotation$Well == well, 4])
}
NK$Group <- as.factor(sapply(NK$Metadata_Well, getGroup))

In [ ]:
getSh <- function(well){ # Which gene is targeted in a given well?
    x = wellAnnotation[wellAnnotation$Well == well, ]
    return(paste(x[2], x[3], sep = "_"))
}
NK$shRNA <- as.factor(sapply(NK$Metadata_Well, getSh))

## Visualize full dataset

### Cell count

In [ ]:
ggplot(NK) + geom_histogram(aes(Count_FilteredNucleus, fill = Plate), binwidth=2)

In [ ]:
ggplot(NK) + geom_point(aes(Count_Nucleus, Count_FilteredNucleus, color = Plate), position = "jitter")

In [ ]:
ggplot(NK) + geom_point(aes(Mean_FilterCytoplasm_AreaShape_Perimeter, Count_FilteredNucleus, 
                            color = Plate), position = "jitter")

## Filtering

In [ ]:
FILT_MAX_INT_DNA = 0.05 # Remove empty images and small DNA precipitations
FILT_MIN_CELLS = 4 # 8 seems safe from distribution and images, 3 seems in poor shape
FILT_NB_MAX_NA_IMAGE = 10 # 48 images generated between 10 and 385 NAs/image, all others generate at most 2/image
FILT_MAX_CORR = 0.6 # Keep uncorrelated variables
# Strong clots on DNA channels are observe to be technical artifacts driving clustering and needs to be removed
FILT_MINMAX_INT_DNA = 0.6
dimUMAP = 3

In [ ]:
ftToKeep = 1:dim(NK)[2]
# Make sure that the fields are numeric
ftToKeep <- ftToKeep[which(sapply(NK[,ftToKeep], class) == "numeric")]
# Remove execution time and count features
ftToKeep <- ftToKeep[grep("(Execution)|(Count)", colnames(NK)[ftToKeep], invert = T)]

In [ ]:
# Store all remaining features before filtering for downstream comparison to selected set
preFiltFt = colnames(NK)[ftToKeep]

In [ ]:
# Remove wells with low max DNA intensity
fieldToKeep <- which(NK$ImageQuality_MaxIntensity_DNA >= FILT_MAX_INT_DNA)
# Remove wells with high max DNA intensity
fieldToKeep <- which(NK$ImageQuality_MaxIntensity_DNA <= FILT_MINMAX_INT_DNA)
# Remove wells with low cell count
fieldToKeep <- fieldToKeep[NK[fieldToKeep,]$Count_FilteredNucleus >= FILT_MIN_CELLS]

In [ ]:
# Few bad quality pictures are generating a lot of missing values and are removed
fieldToKeep <- fieldToKeep[rowSums(is.na(NK[fieldToKeep,ftToKeep])) < FILT_NB_MAX_NA_IMAGE]
# Remove remaining features with missing values
ftToKeep <- ftToKeep[colSums(is.na(NK[fieldToKeep,ftToKeep])) == 0] 
# Remove constant columns
indWT = NK[fieldToKeep,]$Gene == "WT"
ftToKeep <- ftToKeep[sapply(NK[fieldToKeep,ftToKeep], function(x) mad(x) != 0)]
ftToKeep <- ftToKeep[sapply(NK[fieldToKeep[indWT],ftToKeep], function(x) mad(x) != 0)]

In [ ]:
# Approximate normal distribution
transfLog <- function (x){
    minVal = min(x)
    return(log(x+1-minVal))
}
transformedNK = as.data.frame(apply(NK[fieldToKeep, ftToKeep], 2, transfLog))

In [ ]:
# Center and scale on control values
transfNorm <- function(x, y){
    return((x - median(y))/mad(y))
}
for (plate in levels(NK$Plate)){
    indPlateField = NK$Plate[fieldToKeep] == plate
    indWTinPlate <- NK[fieldToKeep[indPlateField],]$Gene == "WT"
    
    transformedNK[indPlateField,] = apply(
        transformedNK[indPlateField,], 2, 
        function(x) transfNorm(x, x[indWTinPlate]))
}

In [ ]:
# Remove columns with NA, i.e.
# features with mad == 0 for 1 plate or more
noNAFt = colSums(is.na(transformedNK)) == 0
ftToKeep = ftToKeep[noNAFt]
transformedNK = transformedNK[,noNAFt]

In [ ]:
# Order features from biggest mad to smallest mad
# Since features have mad(WT) = 1 on each plate, it means that we rank features by how more variable they are
# for drug perturbations than for WT
orderFt = rev(order(apply(transformedNK, 2, mad)))

In [ ]:
uncorrFt = uncorrelate(transformedNK, orderCol = orderFt-1, threshold = FILT_MAX_CORR)
uncorrFt = unlist(uncorrFt) + 1

In [ ]:
transformedNK = transformedNK[,uncorrFt]

In [ ]:
dim(transformedNK)

### Look at which types of features are kept

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrPerf")
table(rowSums(sapply(catChannel, function(x) grepl(x, colnames(transformedNK)))))
dtCat = data.frame(CountIni = sort(colSums(sapply(catChannel, function(x) grepl(x, preFiltFt)))), 
                   Count = sort(colSums(sapply(catChannel, function(x) grepl(x, colnames(transformedNK))))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cemm_pal(2)[2]) + ylim(c(0,675)) +
              geom_bar(aes(Category, weight = Count), fill = cemm_pal(2)[1]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_SelecFt_Channel.pdf", plot = gp, width = 10)
}

In [ ]:
catObjects = c("ActinGranules", "FilterCytoplasm", "ShrunkenCytoplasm", "FilteredNucleus", "PerfGranules")
table(rowSums(sapply(catObjects, function(x) grepl(x, colnames(transformedNK)))))
dtCat = data.frame(CountIni = colSums(sapply(catObjects, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catObjects, function(x) grepl(x, colnames(transformedNK)))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cemm_pal(2)[2]) + ylim(c(0,510)) +
              geom_bar(aes(Category, weight = Count), fill = cemm_pal(2)[1]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_SelecFt_Object.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Threshold", "Granularity", "ImageQuality", "Texture", "Distance", "AreaShape", "RadialDistribution", "Neighbors", 
            "Correlation", "Intensity", "Overlap", "Location")
which(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedNK)))) == 0) # All features are covered

dtCat = data.frame(CountIni = colSums(sapply(catType, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catType, function(x) grepl(x, colnames(transformedNK)))))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cemm_pal(2)[2]) +
              geom_bar(aes(Category, weight = Count), fill = cemm_pal(2)[1]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_SelecFt_Type_All.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
table(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedNK)))))

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedNK))))
dtCount$Other = !apply(dtCount, 1, any)
dtCountIni = as.data.frame(sapply(catType, function(x) grepl(x, preFiltFt)))
dtCountIni$Other = !apply(dtCountIni, 1, any)
dtCat = data.frame(CountIni = colSums(dtCountIni), 
                   Count = colSums(dtCount))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cemm_pal(2)[2]) +
              geom_bar(aes(Category, weight = Count), fill = cemm_pal(2)[1]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_SelecFt_Type_Short.pdf", plot = gp, width = 10)
}

### Look at the morphological distribution of the fields of view

In [ ]:
# Fix t-SNE layout and random number generation
set.seed(38)

In [ ]:
umTNK = umap(transformedNK, min_dist = 0.1, neighbors = 10, n = dimUMAP, metric = "euclidean")
umTNK = as.data.frame(umTNK)
names(umTNK) = paste0("UMAP", 1:dimUMAP)

In [ ]:
umTNK$Row <- as.factor(NK$Metadata_Row[fieldToKeep])
umTNK$Col <- as.factor(NK$Metadata_Column[fieldToKeep])
umTNK$URL <- as.factor(NK$URL_Actin[fieldToKeep])
umTNK$Gene <- as.factor(NK$Gene[fieldToKeep])
umTNK$shRNA <- as.factor(NK$shRNA[fieldToKeep])
umTNK$Group <- as.factor(NK$Group[fieldToKeep])

In [ ]:
gp <- ggplot(umTNK) + geom_point(aes(UMAP1, UMAP2, color = Gene))
gp + geom_label_repel(aes(UMAP1, UMAP2), 
              label = ifelse((umTNK$UMAP2 < -15), str_extract(as.character(umTNK$URL), "r..c..f.."), NA),
              size = 1.5, segment.alpha = 0.1)

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_UMAP_Genes.pdf", plot = gp, width = 10)
}

In [ ]:
gp <- ggplot(umTNK) + geom_point(aes(UMAP1, UMAP2, color = Group))
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_UMAP_Group.pdf", plot = gp, width = 10)
}

In [ ]:
gp <- ggplot(umTNK) + geom_point(aes(UMAP1, UMAP2, color = ifelse(Gene == "WT", "WT", ifelse(Gene == "NOTARGET", "No target", "shRNA")))) +
    scale_color_discrete("Target")
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_UMAP_WT.pdf", plot = gp, width = 10)
}

In [ ]:
# Compute the median Robust Mahalanobis Distance (RMD) between drugs
shRMD <- function(shPert){
    # Find row containing this drug and split wells between WT and drug
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$shRNA == shPert],]$Metadata_Row[1]
    setUmapSh = umTNK[NK[fieldToKeep,]$shRNA == shPert,1:dimUMAP]
    setUmapWT = umTNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    shPlates = NK$Plate[fieldToKeep[NK[fieldToKeep,]$shRNA == shPert]]
    wtPlates = NK$Plate[fieldToKeep[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert]]]

    plateRMD <- function(plate){
        subsetUmapSh = setUmapSh[shPlates == plate,]
        subsetUmapWT = setUmapWT[wtPlates == plate,]
        if ((dim(subsetUmapSh)[1] < 2*dimUMAP)|(dim(subsetUmapWT)[1] < 2*dimUMAP)){
            print(c(shPert, plate))
            return(NA)
        }
        
        # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
        mcdWT = covMcd(subsetUmapWT)
        RMD = apply(subsetUmapSh, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov))

        return(RMD)
    }

    # For all plates including perturbation
    RMD = median(unlist(sapply(levels(NK[fieldToKeep,]$Plate[NK[fieldToKeep,]$shRNA == shPert]), plateRMD)), na.rm = T)
    return(RMD)
}

In [ ]:
s = Sys.time()
levelSH = levels(NK$shRNA)[!grepl("WT", levels(NK$shRNA))]
shRMD_NK = sapply(levelSH, shRMD)
print(Sys.time() - s)

In [ ]:
na.omit.list <- function(y) { return(y[!sapply(y, function(x) all(is.na(x)))]) }

shuffShRMD <- function(shPert, nbRep = PERM_NB_ITER){
    # Find row containing this drug and split wells between WT and drug
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$shRNA == shPert],]$Metadata_Row[1]
    setUmapSh = umTNK[NK[fieldToKeep,]$shRNA == shPert,1:dimUMAP]
    setUmapWT = umTNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],1:dimUMAP]
    shPlates = NK$Plate[fieldToKeep[NK[fieldToKeep,]$shRNA == shPert]]
    wtPlates = NK$Plate[fieldToKeep[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert]]]
    
    plateRMD <- function(plate){
        subsetUmapSh = setUmapSh[shPlates == plate,]
        subsetUmapWT = setUmapWT[wtPlates == plate,]
        if ((dim(subsetUmapSh)[1] < 2*dimUMAP)|(dim(subsetUmapWT)[1] < 2*dimUMAP)){
            return(rep(NA, nbRep))
        }
        setUMAP = rbind(subsetUmapSh, subsetUmapWT)

        shuffleRMD <- function(notUsed){
            shuffSetUMAP = setUMAP[sample(nrow(setUMAP)),]
            # Take random subsets of corresponding sizes
            shuffSetSh = shuffSetUMAP[1:nrow(subsetUmapSh),]
            shuffSetWT = shuffSetUMAP[(nrow(subsetUmapSh)+1):(nrow(subsetUmapSh)+nrow(subsetUmapWT)),]
            # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
            mcdWT = covMcd(shuffSetWT)
            RMD = apply(shuffSetSh, 1, function(x) mahalanobis(x, mcdWT$center, mcdWT$cov))
            if(any(RMD < 0)){
                print(c(shPert, plate, notUsed, dim(shuffSetWT)))
                print(RMD)
                print(mcdWT$cov)
                print(det(mcdWT$cov))
            }
            return(RMD)
        }
        return(unlist(sapply(1:nbRep, shuffleRMD)))
    }

    # For all plates including perturbation
    intTemp = sapply(levels(NK[fieldToKeep,]$Plate[NK[fieldToKeep,]$shRNA == shPert]), plateRMD)
    if(all(is.na(intTemp))){
        # No plate can be used for the perturbation
        return(rep(NA, nbRep))
    }
    if(class(intTemp) == "matrix"){
        # All plates complete, results get simplified to a matrix (column = plate)
        RMD = aggregate(matrix(intTemp), 
                        by = list(rep(rep(1:nbRep, each = dim(intTemp)[1]/nbRep), 3)), median)$V1
    }
    else{
        # We have a list for each plate containing a matrix of RMDs
        # Columns = replicate, rows = wells
        intTemp = na.omit.list(intTemp) # Remove NAs from perturbations with unsufficient well counts
        intTemp = do.call(rbind, Map(data.frame, intTemp)) # Assemble into a single data.frame
        RMD = colMedians(as.matrix(intTemp), na.rm = T) # Compute medians for each simulation
    }
    return(unname(RMD))
}

In [ ]:
if(TEST_MODE){
    s = Sys.time()
    shuff_shRMD_NK = sapply(levelSH, function(x) shuffShRMD(x, nbRep = 2000))
    print(Sys.time() - s)
}

In [ ]:
shRMD_NK

In [ ]:
head(shuff_shRMD_NK)

In [ ]:
table(is.na(shRMD_NK))
table(is.na(shuff_shRMD_NK))

In [ ]:
getRMPV <- function(x){
    if(is.na(x[1])){
        return(NA)
    }
    ecdfRMD = ecdf(x[-1])
    return(1 - ecdfRMD(x[1]))
}
dfRMPV = data.frame(RMPV = apply(rbind(shRMD_NK, shuff_shRMD_NK), 2, getRMPV))

In [ ]:
dfRMPV$shRNA = rownames(dfRMPV)
dfRMPV$Strength = shRMD_NK
dfRMPV$adjRMPV = p.adjust(dfRMPV$RMPV, method = "fdr")
dfRMPV$labelOut = ifelse((dfRMPV$adjRMPV > 0.05)|(is.na(dfRMPV$adjRMPV)), dfRMPV$shRNA, '')
dfRMPV$labelIn = ifelse(!((dfRMPV$adjRMPV > 0.05)|(is.na(dfRMPV$adjRMPV))), dfRMPV$shRNA, '')

In [ ]:
gp <- ggplot(dfRMPV) + geom_point(aes(adjRMPV, Strength, color = shRNA)) + geom_vline(xintercept = 0.05, color="#CCCCCC", linetype="dashed") +
                    theme(legend.position="none") + geom_label_repel(aes(adjRMPV, Strength, label = labelOut))
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_RMPV.pdf", gp, width = 10)
}

### Morphological changes upon perturbation

In [ ]:
# Associate categories and colors to features
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Intensity")
colType = cemm_pal(length(catType)+1)

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedNK))))
dtCount$Other = !apply(dtCount, 1, any)
 
ftCat = as.factor(apply(dtCount, 1, function(x) colnames(dtCount)[which(x)]))
ftCol = colType[ftCat]

In [ ]:
# Plot number of images / categories (useful to get fill color legend)
gp <- ggplot(data.frame(Category = ftCat), aes(fill = Category)) + geom_bar(aes(Category))
if(!TEST_MODE){
    ggsave(filename = "Fig/SH_NK_CatCount.pdf", plot = gp, width = 10)
}

In [ ]:
# Which drugs should be studied?
sigSh = unique(dfRMPV$labelIn)

In [ ]:
changedFtDrug <- function(shPert){  
    # Plate row on which the drug is
    rowPert = NK[fieldToKeep[NK[fieldToKeep,]$shRNA == shPert],]$Metadata_Row[1]
    
    # Morphological features for the drug and the WT control on the same row
    setSh = transformedNK[NK[fieldToKeep,]$shRNA == shPert,]
    setWT = transformedNK[which(indWT)[NK[fieldToKeep[indWT],]$Metadata_Row == rowPert],]
    
    medianSh = apply(setSh, 2, median)
    medianWT = apply(setWT, 2, median)
    medianChange = medianSh - medianWT
    
    # Compute difference to WT
    ftToDisplay = order(abs(medianChange), decreasing = T)[1:12]
    dtChange = data.frame(Difference = medianChange[ftToDisplay])
    dtChange$Feature = rownames(dtChange)
    dtChange$Feature = factor(dtChange$Feature, levels = dtChange$Feature[order(dtChange$Difference)]) 
    dtChange$Category = ftCol[ftToDisplay]

    gp <- ggplot(dtChange) + geom_bar(aes(Feature, weight = Difference, fill = Category), color = "#CCCCCC") + 
                   coord_flip() + ylab("Difference to WT") + scale_fill_identity() +
                   theme(legend.position="none", plot.margin=unit(c(0,5,0,0), "mm"))
    
    if(!TEST_MODE){
        ggsave(filename = paste("Fig/SH_NK", shPert, "ChangeFt.pdf", sep = "_"), plot = gp, width = 10)
    }
}

In [ ]:
sapply(sigSh, changedFtDrug)

In [ ]:
sessionInfo()